In [1]:
import urllib.request
import zipfile
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [2]:
def solution_A3():
    inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
    urllib.request.urlretrieve(
        inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
    local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

    pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                                    include_top=False,
                                    weights=None)
    pre_trained_model.load_weights(local_weights_file)

    for layer in pre_trained_model.layers:
        layer.trainable = False

    last_layer = pre_trained_model.get_layer('mixed7')
    last_output = last_layer.output

    data_url_1 = 'https://github.com/dicodingacademy/assets/releases/download/release-horse-or-human/horse-or-human.zip'
    urllib.request.urlretrieve(data_url_1, 'horse-or-human.zip')
    local_file = 'horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/horse-or-human')
    zip_ref.close()

    data_url_2 = 'https://github.com/dicodingacademy/assets/raw/main/Simulation/machine_learning/validation-horse-or-human.zip'
    urllib.request.urlretrieve(data_url_2, 'validation-horse-or-human.zip')
    local_file = 'validation-horse-or-human.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/validation-horse-or-human')
    zip_ref.close()

    train_dir = 'data/horse-or-human'
    validation_dir = 'data/validation-horse-or-human'

    train_datagen = ImageDataGenerator(rescale=1.0/255.0)
    validation_datagen = ImageDataGenerator(rescale=1.0/255.0)

    # YOUR IMAGE SIZE SHOULD BE 150x150
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

    validation_generator = validation_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='binary')

    x = layers.Flatten()(last_output)
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(1, activation='sigmoid')(x)

    model = Model(pre_trained_model.input, x)

    model.compile(optimizer=RMSprop(lr=0.0001),
                  loss='binary_crossentropy',
                  metrics=['acc'])

    train_steps_per_epoch = train_generator.samples // train_generator.batch_size
    validation_steps = validation_generator.samples // validation_generator.batch_size

    model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=10,
        steps_per_epoch=train_steps_per_epoch,
        validation_steps=validation_steps,
        verbose=2)

    return model

In [3]:
# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_A3()
    model.save("model_A3.h5")

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.
Epoch 1/10
51/51 - 17s - loss: 0.3826 - acc: 0.9652 - val_loss: 2.1358e-04 - val_acc: 1.0000 - 17s/epoch - 341ms/step
Epoch 2/10
51/51 - 11s - loss: 1.4086e-05 - acc: 1.0000 - val_loss: 0.0014 - val_acc: 1.0000 - 11s/epoch - 223ms/step
Epoch 3/10
51/51 - 11s - loss: 6.4509e-08 - acc: 1.0000 - val_loss: 1.6649e-06 - val_acc: 1.0000 - 11s/epoch - 224ms/step
Epoch 4/10
51/51 - 11s - loss: 1.1349e-08 - acc: 1.0000 - val_loss: 0.0015 - val_acc: 1.0000 - 11s/epoch - 225ms/step
Epoch 5/10
51/51 - 11s - loss: 6.1046e-08 - acc: 1.0000 - val_loss: 0.0016 - val_acc: 1.0000 - 11s/epoch - 224ms/step
Epoch 6/10
51/51 - 11s - loss: 9.7295e-08 - acc: 1.0000 - val_loss: 0.0019 - val_acc: 1.0000 - 11s/epoch - 224ms/step
Epoch 7/10
51/51 - 11s - loss: 6.8844e-08 - acc: 1.0000 - val_loss: 0.0021 - val_acc: 1.0000 - 11s/epoch - 223ms/step
Epoch 8/10
51/51 - 11s - loss: 9.4362e-08 - acc: 1.0000 - val_loss: 0.0026 - val_acc: 1

c:\Users\ilham\miniconda3\envs\vision1\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
